# Serving a Model as a REST Endpoint

![SageMaker Endpoint Architecture](img/sagemaker-architecture.png)

In [1]:
!pip install -q --upgrade pip
!pip install -q wrapt --upgrade --ignore-installed
!pip install -q tensorflow==2.1.0
!pip install -q transformers==2.8.0

In [2]:
!pip install --upgrade sagemaker==1.56.1

Requirement already up-to-date: sagemaker==1.56.1 in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (1.56.1)


In [3]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [4]:
%store -r training_job_name

no stored variable training_job_name


In [5]:
print(training_job_name)

NameError: name 'training_job_name' is not defined

# Copy the Model to the Notebook

In [ ]:
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz ./model.tar.gz

In [ ]:
!tar -xvzf ./model.tar.gz

In [ ]:
!saved_model_cli show --all --dir ./tensorflow/saved_model/0/

# Deploy the Model
This will create a default `EndpointConfig` with a single model.  

The next notebook will demonstrate how to perform more advanced `EndpointConfig` strategies to support canary rollouts and A/B testing.

_Note:  If not using a US-based region, you may need to adapt the container image to your current region using the following table:_

https://docs.aws.amazon.com/deep-learning-containers/latest/devguide/deep-learning-containers-images.html

In [ ]:
from sagemaker.tensorflow.serving import Model

model = Model(model_data='s3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name),
              role=role,
              # Note:  `image` is required because the inference and training images are different.
              #        By default, the training image will be used, so we need to override it.  
              #        See https://github.com/aws/sagemaker-python-sdk/issues/1379
#              image='763104351884.dkr.ecr.{}.amazonaws.com/tensorflow-inference-eia:2.0.0-cpu-py36-ubuntu18.04'.format(region),
              framework_version='2.0.0') # Elastic Inference does not yet support TF 2.1.0 as of sagemaker==1.56.1

In [ ]:
deployed_model = model.deploy(initial_instance_count=2, # Minimum 2 for high(er) availability 
                              instance_type='ml.m4.xlarge',
                              accelerator_type='ml.eia2.medium',
                              wait=False)

endpoint_name = deployed_model.endpoint

print('Endpoint name:  {}'.format(endpoint_name))


In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">REST Endpoint</a></b>'.format(region, endpoint_name)))


# Store Endpoint Name for Next Notebook(s)

In [ ]:
%store endpoint_name

# _Wait Until the ^^ Endpoint ^^ is Deployed_

In [ ]:
client = boto3.client('sagemaker')
waiter = client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

# Simulate a Prediction from an Application

In [ ]:
class RequestHandler(object):
    import json
    
    def __init__(self, tokenizer, max_seq_length):
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length

    def __call__(self, instances):
        transformed_instances = []

        for instance in instances:
            encode_plus_tokens = tokenizer.encode_plus(instance,
                                                       pad_to_max_length=True,
                                                       max_length=self.max_seq_length)

            input_ids = encode_plus_tokens['input_ids']
            input_mask = encode_plus_tokens['attention_mask']
            segment_ids = [0] * MAX_SEQ_LENGTH

            transformed_instance = {"input_ids": input_ids, 
                                    "input_mask": input_mask, 
                                    "segment_ids": segment_ids}

            transformed_instances.append(transformed_instance)

        transformed_data = {"instances": transformed_instances}

        return json.dumps(transformed_data)

In [ ]:
class ResponseHandler(object):
    import json
    import tensorflow as tf
    
    def __init__(self, classes):
        self.classes = classes
    
    def __call__(self, response, accept_header):
        import tensorflow as tf

        response_body = response.read().decode('utf-8')

        response_json = json.loads(response_body)

        log_probabilities = response_json["predictions"]

        predicted_classes = []

        # Convert log_probabilities => softmax (all probabilities add up to 1) => argmax (final prediction)
        for log_probability in log_probabilities:
            softmax = tf.nn.softmax(log_probability)    
            predicted_class_idx = tf.argmax(softmax, axis=-1, output_type=tf.int32)
            predicted_class = self.classes[predicted_class_idx]
            predicted_classes.append(predicted_class)

        return predicted_classes

In [ ]:
import json
from sagemaker.tensorflow.serving import Predictor
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

request_handler = RequestHandler(tokenizer=tokenizer,
                                 max_seq_length=128)

response_handler = ResponseHandler(classes=[1, 2, 3, 4, 5])

predictor = Predictor(endpoint_name=endpoint_name,
                      sagemaker_session=sess,
                      serializer=request_handler,
                      deserializer=response_handler,
                      content_type='application/json',
                      model_name='saved_model',
                      model_version=0)

In [ ]:
import tensorflow as tf
import json
    
reviews = ["This is great!", 
           "This is terrible."]

predicted_classes = predictor.predict(reviews)

for predicted_class, review in zip(predicted_classes, reviews):
    print('[Predicted Star Rating: {}]'.format(predicted_class), review)

# Simulate a (Mini-)Load Test

In [ ]:
def predict(idx):
    reviews = ["This is great!", 
               "This is terrible."]

    predicted_classes = predictor.predict(reviews)

    return predicted_classes

In [ ]:
import functools
import multiprocessing

num_cpus = multiprocessing.cpu_count()

p = multiprocessing.Pool(num_cpus)

In [ ]:
%%time

results = p.map(predict, range(1,100))

# TODO:  Verify that Elastic Inference is working
https://docs.aws.amazon.com/sagemaker/latest/dg/ei.html

In [ ]:
# !aws cloudwatch list-metrics --namespace " AWS/ElasticInference "

# Optimize Cost with TensorFlow and Elastic Inference
https://aws.amazon.com/blogs/machine-learning/optimizing-costs-in-amazon-elastic-inference-with-amazon-tensorflow/

# Using API Gateway with SageMaker Endpoints

https://aws.amazon.com/blogs/machine-learning/creating-a-machine-learning-powered-rest-api-with-amazon-api-gateway-mapping-templates-and-amazon-sagemaker/